# Invoke Bedrock model using LangChain and a zero-shot prompt

## Introduction

In this notebook we show how to use a LLM to generate an email response to a customer who provided negative feedback on the quality of customer service that they received from the support engineer. 

We will use Anthropic's Claude model provided by Bedrock in this example. We will use the Bedrock version that is integrated with [LangChain](https://python.langchain.com/docs/get_started/introduction.html). LangChain is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example is called a zero-shot prompt because we are not providing any additional context other than the prompt.

**Note:** *This notebook can be run within or outside of AWS environment*.

#### Context
In the previous example `00_generate_w_bedrock.ipynb`, we explored how to use Boto3 client to communicate with Amazon Bedrock API. In this notebook, we will try to add a bit more complexity to leverage the LangChain framework for the similar use case. We will explore the use of Amazon Bedrock integration within LangChain framework and how it could be used to generate text with the help of `PromptTemplate`.

#### Pattern
We will simply provide the LangChain implementation of Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/bedrock_langchain.jpg)

#### Useccase
To demonstrate the generation capability of models in Amazon Bedrock, let's take the use case of email generation.

#### Persona
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

#### Implementation
To fulfill this use case, in this notebook we will show how to generate an email with a thank you note based on the customer's previous email. We will use the Amazon Titan Text Large model using the Amazon Bedrock LangChain integration. 

## Invoke the Bedrock client using LangChain Integration

Lets begin with creating an instance of Bedrock class from llms. This expects a `model_id` of the model available in Amazon Bedrock. 

Optionally you can pass on a previously created boto3 client as well as some `model_kwargs` which can hold parameters such as `temperature`, `topP`, `maxTokenCount` or `stopSequences` (more on parameters can be explored in Amazon Bedrock console).

Available text generation models under Amazon Bedrock have the following IDs:

- amazon.titan-tg1-large
- ai21.j2-grande-instruct
- ai21.j2-jumbo-instruct
- anthropic.claude-instant-v1
- anthropic.claude-v1

## Setup 

#### ⚠️⚠️⚠️ Execute the following cells before running this notebook ⚠️⚠️⚠️

For a detailed description on what the following cells do refer to [Bedrock boto3 setup](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In [ ]:
# Make sure you run `download-dependencies.sh` from the root of the repository to download the dependencies before running this cell
%pip install ../dependencies/botocore-1.29.162-py3-none-any.whl ../dependencies/boto3-1.26.162-py3-none-any.whl ../dependencies/awscli-1.27.162-py3-none-any.whl --force-reinstall
%pip install langchain==0.0.190 --quiet

In [ ]:
%pip install langchain==0.0.190 --quiet

In [ ]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [ ]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
boto3_bedrock = bedrock.get_bedrock_client(os.environ.get('BEDROCK_ASSUME_ROLE', None))

Amazon Bedrock API can be used with these parameters below:
- `inputTextTokenCount`: The size of the input prompt
- `tokenCount`: The number of tokens of the entire prompt + the output

It will return the results as below:
- `outputText`: The text generated by the model

In [ ]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v1",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )

LangChain has abstracted away the Amazon Bedrock API and made it easy to build use cases. You can pass in your prompt and it is automatically routed to the appropriate API to generate the response. You simply get the text output as-is and don't have to extract the results out of the response body.

Let's prepare the prompt to generate an email for the Customer Service Manager to send to the customer.

In [ ]:
response = textgen_llm("""Write an email from Bob, Customer Service Manager, 
to the customer "John Doe" that provided negative feedback on the service 
provided by our customer support engineer.\n\nHuman:""")

print_ww(response)

## Conclusion
You have now experimented with using `LangChain` framework which provides an abstraction layer on Amazon Bedrock API. Using this framework you have seen the usecase of generating an email responding to a customer due to their negative feedback.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Anthropic Claude and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the different parameters to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You